In [1]:
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, set_seed
import evaluate
import numpy as np
import os
import pandas as pd
import random
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
set_seed(42)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

train_set = pd.read_csv('./data/letterboxd_250movie_reviews_train.csv')
val_set  = pd.read_csv('./data/letterboxd_250movie_reviews_val.csv')
test_set  = pd.read_csv('./data/letterboxd_250movie_reviews_test.csv')

dataset = DatasetDict({
    'train': Dataset.from_pandas(train_set),
    'validation': Dataset.from_pandas(val_set),
    'test': Dataset.from_pandas(test_set)
})

model_path = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {
    0: "½",
    1: "★",
    2: "★½",
    3: "★★",
    4: "★★½",
    5: "★★★",
    6: "★★★½",
    7: "★★★★",
    8: "★★★★½",
    9: "★★★★★",
}
label2id = {
    "½": 0,
    "★": 1,
    "★½": 2,
    "★★": 3,
    "★★½": 4,
    "★★★": 5,
    "★★★½": 6,
    "★★★★": 7,
    "★★★★½": 8,
    "★★★★★": 9,
}
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=10,
    id2label=id2label,
    label2id=label2id,
)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_data = dataset.map(preprocess_function, batched=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [2]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [3]:
# hyperparameters
lr = 2e-4
batch_size = 16
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-letterbox-reviews-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


In [4]:
tokenized_data["validation"]

Dataset({
    features: ['original_id', 'text', 'label', 'movie', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
for name, param in model.base_model.named_parameters():
    param.requires_grad = False
    
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

# for name, param in list(model.named_parameters())[20:]:
#     if "layer.11." in name:
#         param.requires_grad = True

In [6]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc", "multiclass")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred
    
    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # compute multiclass auc using all class probabilities
    auc = np.round(auc_score.compute(prediction_scores=probabilities, references=labels, multi_class='ovr')['roc_auc'],3)
    
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

training_output = trainer.train()

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_64919/2642840178.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,2.214800,2.135595,0.185000,0.712000
2,1.905700,1.914915,0.270000,0.764000
3,1.682100,1.966544,0.215000,0.763000
4,1.439400,2.052332,0.225000,0.763000
5,1.158800,2.310039,0.230000,0.744000
6,0.864700,2.425236,0.240000,0.744000
7,0.609900,2.638079,0.200000,0.737000
8,0.413100,2.904823,0.190000,0.724000
9,0.295700,2.914905,0.185000,0.717000
10,0.234000,2.940443,0.200000,0.718000


/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on 

In [ ]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["test"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))

# pool layer unfrozen
# 200 per rating baseline: {'Accuracy': np.float64(0.1), 'AUC': np.float64(0.536)}
# 200 per rating, {'Accuracy': np.float64(0.265), 'AUC': np.float64(0.738)} lr = 2e-4 batch_size = 16 num_epochs = 10
# 300 per rating baseline: {'Accuracy': np.float64(0.114), 'AUC': np.float64(0.526)}
# 300 per rating, {'Accuracy': np.float64(0.193), 'AUC': np.float64(0.716)} lr = 2e-4 batch_size = 16 num_epochs = 10
# 300 per rating, {'Accuracy': np.float64(0.200), 'AUC': np.float64(0.716)} lr = 2e-4 batch_size = 8 num_epochs = 10
# 300 per rating, {'Accuracy': np.float64(0.154), 'AUC': np.float64(0.663)} lr = 2e-5 batch_size = 8 num_epochs = 10
# 300 per rating, {'Accuracy': np.float64(0.214), 'AUC': np.float64(0.716)} lr = 2e-4 batch_size = 8 num_epochs = 20
# 200 per rating, {'Accuracy': np.float64(0.265), 'AUC': np.float64(0.745)} lr = 2e-4 batch_size = 8 num_epochs = 20
# 200 per rating, {'Accuracy': np.float64(0.275), 'AUC': np.float64(0.745)} lr = 2e-4 batch_size = 16 num_epochs = 20
# 200 per rating, {'Accuracy': np.float64(0.180), 'AUC': np.float64(0.677)} lr = 2e-5 batch_size = 16 num_epochs = 20
print(metrics)

# also layer 11 unfrozen
# 200 per rating, {'Accuracy': np.float64(0.23), 'AUC': np.float64(0.776)} lr = 2e-4 batch_size = 8 num_epochs = 10
# 200 per rating, {'Accuracy': np.float64(0.31), 'AUC': np.float64(0.785)} lr = 2e-4 batch_size = 16 num_epochs = 10


/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'Accuracy': np.float64(0.31), 'AUC': np.float64(0.785)}


In [9]:
import torch
def predict_single(review: str):
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = trainer.predict(Dataset.from_dict(inputs))
        predicted_class_id = outputs.predictions.argmax().item()

        return model.config.id2label[predicted_class_id]

def predict_and_print(review: str):
    prediction = predict_single(review)
    print(f"{review} -> {prediction}")
predict_and_print("a whole movie about coming down from apex mountain. Safdie's worthy, long-overdue follow-up to Lenny Cooke")
predict_and_print("the rock is evolving. he is now the boulder")
predict_and_print("That guy with the long hair was so unnecessarily rude.")
predict_and_print("utterly sauceless")
predict_and_print("decent as a (very) muted romance, much louder and more effective as an argument for physical archival media.")
predict_and_print("That was incredibly upsetting but I knew it would be")
predict_and_print("you’re not depressed, you just love new england")

/Users/miguelvilagonzalez/repos/ai-builders-week-5/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


a whole movie about coming down from apex mountain. Safdie's worthy, long-overdue follow-up to Lenny Cooke -> ★★★★★


the rock is evolving. he is now the boulder -> ★★★★★


That guy with the long hair was so unnecessarily rude. -> ½


utterly sauceless -> ½


decent as a (very) muted romance, much louder and more effective as an argument for physical archival media. -> ★½


That was incredibly upsetting but I knew it would be -> ★★★★★


you’re not depressed, you just love new england -> ★★★★½
